# 🧬 Animated Beings

## Explore carousels for various living categories below. Each section features interactive image selectors built using Dash.

### Human Carousel

In [1]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Dash config with parent asset path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🧍 Human Face Carousel"

# 📦 Paths
HUMAN_PATH = os.path.abspath("../assets/human")
HUMAN_PARTS = sorted([d for d in os.listdir(HUMAN_PATH) if os.path.isdir(f"{HUMAN_PATH}/{d}")])

# 📦 Face-specific subfolders
FACE_SEX = ["female", "male"]
FACE_AGES = ["baby", "child", "adolescent", "adult", "elder"]

# 📦 Load non-face images
basic_images = {
    part: sorted([
        f"/assets/human/{part}/{img}"
        for img in os.listdir(os.path.join(HUMAN_PATH, part))
        if img.lower().endswith((".jpg", ".jpeg", ".png"))
    ])
    for part in HUMAN_PARTS if part != "face"
}

# 📦 Load face images
face_images = {}
for sex in FACE_SEX:
    for age in FACE_AGES:
        folder = os.path.join(HUMAN_PATH, "face", sex, age)
        if os.path.isdir(folder):
            face_images[(sex, age)] = sorted([
                f"/assets/human/face/{sex}/{age}/{img}"
                for img in os.listdir(folder)
                if img.lower().endswith((".jpg", ".jpeg", ".png"))
            ])

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🧍 Human Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "24px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select body part:", style={
            "fontWeight": "normal", "display": "block", "marginBottom": "6px",
            "fontSize": "15px", "fontFamily": "Georgia, serif"
        }),
        dcc.Dropdown(
            id='part-dropdown',
            options=[{"label": part.capitalize(), "value": part} for part in HUMAN_PARTS],
            value=HUMAN_PARTS[0] if HUMAN_PARTS else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='sub-dropdowns', style={"textAlign": "center", "marginBottom": "20px"}),
    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Update sex/age dropdowns if 'face'
@app.callback(
    Output('sub-dropdowns', 'children'),
    Input('part-dropdown', 'value')
)
def update_sub_dropdowns(part):
    if part == "face":
        return html.Div([
            html.Label("Sex:", style={"marginBottom": "6px", "display": "block"}),
            dcc.Dropdown(
                id='sex-dropdown',
                options=[{"label": s.capitalize(), "value": s} for s in FACE_SEX],
                value=FACE_SEX[0],
                clearable=False,
                style={"width": "200px", "margin": "0 auto 12px auto"}
            ),
            html.Label("Age group:", style={"marginBottom": "6px", "display": "block"}),
            dcc.Dropdown(
                id='age-dropdown',
                options=[{"label": a.capitalize(), "value": a} for a in FACE_AGES],
                value=FACE_AGES[0],
                clearable=False,
                style={"width": "200px", "margin": "0 auto"}
            )
        ])
    else:
        return html.Div([
            dcc.Dropdown(id='sex-dropdown', value='female', style={'display': 'none'}),
            dcc.Dropdown(id='age-dropdown', value='baby', style={'display': 'none'})
        ])

# 🔁 Update carousel based on dropdowns
@app.callback(
    Output('carousel-container', 'children'),
    Input('part-dropdown', 'value'),
    Input('sex-dropdown', 'value'),
    Input('age-dropdown', 'value'),
    prevent_initial_call=False
)
def update_carousel(part, sex, age):
    if part == "face":
        images = face_images.get((sex, age), [])
    else:
        images = basic_images.get(part, [])

    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginRight": "15px"}
),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginRight": "15px"}
),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),


        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontStyle": "italic", "fontFamily": "Georgia, serif"})
    ])

# 🔁 Navigation logic
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Run the app
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8167)

### Bird Carousel

In [2]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Tell Dash to look for assets one level up
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)

app.title = "🕊️ Bird Carousel"

# 📦 Liste des espèces d'oiseaux
BIRD_SPECIES = sorted([
    d for d in os.listdir("../assets/birds")
    if os.path.isdir(os.path.join("../assets/birds", d))
])

# 📦 Dictionnaire {species: [images]}
bird_images = {}
for species in BIRD_SPECIES:
    folder = os.path.join("../assets/birds", species)
    images = []
    for root, _, files in os.walk(folder):
        images += [os.path.join(root, f) for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    bird_images[species] = sorted(images)

# 🎨 Layout principal
app.layout = html.Div([
    html.H2("\ud83d\udd4a\ufe0f Bird Carousel", style={
        "textAlign": "center", "marginBottom": "10px", "marginTop": "5px",
        "fontSize": "18px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select bird species:", style={"fontSize": "16px", "marginBottom": "6px", "fontFamily": "Georgia, serif"}),
        dcc.Dropdown(
            id='species-dropdown',
            options=[{"label": s.capitalize(), "value": s} for s in BIRD_SPECIES],
            value=BIRD_SPECIES[0],
            clearable=False,
            style={"width": "300px", "margin": "0 auto", "fontFamily": "Georgia, serif"}
        )
    ], style={"textAlign": "center", "marginBottom": "30px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
], style={"fontFamily": "Georgia, serif", "padding": "30px"})


# 🔄 Affiche le bon carousel pour chaque espèce
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value')
)
def update_carousel(species):
    images = bird_images.get(species, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])

    return html.Div([
        html.Div([
            html.Button("\u2b05\ufe0f", id="prev-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginRight": "15px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "416px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("\u27a1\ufe0f", id="next-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginLeft": "15px"}),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "13px", "fontFamily": "Georgia, serif", "color": "#444"})
    ])

# 🔄 Navigation entre les images
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Lancer 
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8139)

### Fish Carousel

In [3]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Dash config with parent asset path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🐟 Fish Carousel"

# 📦 Liste des espèces
FISH_SPECIES = sorted([
    d for d in os.listdir("../assets/fish")
    if os.path.isdir(f"../assets/fish/{d}")
])

# 📦 Dictionnaire {species: [image paths]}
fish_images = {}
for species in FISH_SPECIES:
    folder = f"../assets/fish/{species}"
    images = []
    for root, _, files in os.walk(folder):
        images += [
            os.path.abspath(os.path.join(root, f))
            for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
    fish_images[species] = sorted(images)

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🐟 Fish Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "24px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select fish species:", style={
            "marginBottom": "6px", "frontSize": "15px", "frontStyle": "normal"
            
        }),
        dcc.Dropdown(
            id='species-dropdown',
            options=[{"label": s.capitalize(), "value": s} for s in FISH_SPECIES],
            value=FISH_SPECIES[0],
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),


    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Affiche carousel
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value')
)
def update_carousel(species):
    images = fish_images.get(species, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    relative_path = os.path.relpath(images[0], os.path.abspath("../assets"))
    dash_path = f"/assets/{relative_path}"

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginRight": "15px"}),
            html.Img(id="carousel-image", src=dash_path,
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),

            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginLeft": "15px"}),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),

        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "13px", "fontFamily": "Georgia, serif"})
    ])

# 🔁 Navigation
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    relative_path = os.path.relpath(images[index], os.path.abspath("../assets"))
    return f"/assets/{relative_path}", index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Launch
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8805)

### Reptile Carousel

In [4]:
from dash import dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Configure app with proper asset path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🦎 Reptile Carousel"

# 📂 Load reptile species
REPTILE_DIR = os.path.abspath("../assets/reptiles")
REPTILE_SPECIES = sorted([
    d for d in os.listdir(REPTILE_DIR)
    if os.path.isdir(os.path.join(REPTILE_DIR, d))
])

# 📸 Dictionary of {species: [image paths]}
reptile_images = {}
for species in REPTILE_SPECIES:
    folder = os.path.join(REPTILE_DIR, species)
    images = []
    for root, _, files in os.walk(folder):
        rel_root = os.path.relpath(root, os.path.abspath("../assets"))
        images += [f"/assets/{rel_root}/{f}" for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    reptile_images[species] = sorted(images)

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🦎 Reptile Carousel", style={"textAlign": "center", "marginBottom": "30px"}),

    html.Div([
        dcc.Dropdown(
            id='species-dropdown',
            options=[{"label": s.replace("_", " ").capitalize(), "value": s} for s in REPTILE_SPECIES],
            value=REPTILE_SPECIES[0],
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Display carousel content based on species
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value')
)
def update_carousel(species):
    images = reptile_images.get(species, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginRight": "15px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginLeft": "15px"}),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "13px", "fontStyle": "italic"})
    ])

# 🔁 Handle navigation
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Run app
clear_output(wait=True)
app.run(port=8169)


### Mammal Carousel

In [5]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
import glob
from IPython.display import clear_output  # ✅ to avoid duplicate display in notebook

# ✅ Tell Dash where to find assets
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "Mammal Carousel"

# 📦 Paths
AQUATIC_PATH = "../assets/mammals/aquatic_mammal"
NONAQUATIC_PARENT = "../assets/mammals"

# 📦 Load species lists
aquatic_species = []
non_aquatic_species = []

try:
    if os.path.exists(AQUATIC_PATH):
        aquatic_species = sorted([
            d for d in os.listdir(AQUATIC_PATH)
            if os.path.isdir(os.path.join(AQUATIC_PATH, d))
        ])
except Exception as e:
    print(f"Error loading aquatic species: {e}")

try:
    if os.path.exists(NONAQUATIC_PARENT):
        non_aquatic_species = sorted([
            d for d in os.listdir(NONAQUATIC_PARENT)
            if os.path.isdir(os.path.join(NONAQUATIC_PARENT, d)) and d != "aquatic_mammal"
        ])
except Exception as e:
    print(f"Error loading non-aquatic species: {e}")

# 📦 Load images
mammal_images = {}

def load_images(folder, key_prefix):
    for species in os.listdir(folder):
        species_path = os.path.join(folder, species)
        if not os.path.isdir(species_path):
            continue
        images = sorted([
            os.path.abspath(f) for f in glob.glob(os.path.join(species_path, "**"), recursive=True)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ])
        if images:
            mammal_images[f"{key_prefix}::{species}"] = images

load_images(AQUATIC_PATH, "aquatic")
load_images(NONAQUATIC_PARENT, "non_aquatic")

# 🔑 Default species
default_species_key = list(mammal_images.keys())[0] if mammal_images else None

# 🎯 Dropdown options
dropdown_options = [
    {"label": k.split("::")[1].replace("_", " ").title(), "value": k}
    for k in mammal_images.keys()
]

# --------------------
# Layout
# --------------------
app.layout = html.Div([
    html.H2("🐻 Mammal Carousel", style={"textAlign": "center", "marginBottom": "25px", "marginTop": "10px"}),

    html.Div([
        html.Label("Select species:"),
        dcc.Dropdown(
            id='species-dropdown',
            options=dropdown_options,
            value=default_species_key,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "40px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# -----------------------------
# Carousel rendering logic
# -----------------------------
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value'),
    prevent_initial_call=False
)
def update_carousel(species_key):
    if not species_key or species_key not in mammal_images:
        return html.P("Please select a species.")

    images = mammal_images[species_key]
    if not images:
        return html.P("No images available for this species.")

    filename = os.path.basename(images[0])
    relative_path = os.path.relpath(images[0], os.path.abspath("../assets"))

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0, style={
                "frontSize": "24px",
                "padding": "6px 12px "
            }),
            html.Img(id="carousel-image", src=f"/assets/{relative_path}",
                     style={"maxHeight": "430px", "maxWidth": "400px"}),
            html.Button("➡️", id="next-button", n_clicks=0, style={
                "frontSize": "24px",
                "padding": "6px 12px "
            }),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display')
    ])

# -----------------------------
# Navigation buttons
# -----------------------------
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images available"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    relative_path = os.path.relpath(images[index], os.path.abspath("../assets"))
    return f"/assets/{relative_path}", index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Run
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8899)

### Insect Carousel

In [6]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Dash config with assets path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🐞 Insect Carousel"

# 📦 Liste des espèces
INSECT_FOLDER = os.path.abspath("../assets/insects")
INSECT_SPECIES = sorted([
    d for d in os.listdir(INSECT_FOLDER)
    if os.path.isdir(os.path.join(INSECT_FOLDER, d))
])

# 📦 Dictionnaire {species: [image list]}
insect_images = {}
for species in INSECT_SPECIES:
    species_path = os.path.join(INSECT_FOLDER, species)
    images = []
    for root, _, files in os.walk(species_path):
        for f in files:
            if f.lower().endswith((".jpg", ".jpeg", ".png")):
                # Get relative path for browser
                rel_path = os.path.relpath(os.path.join(root, f), INSECT_FOLDER)
                web_path = f"/assets/insects/{rel_path.replace(os.sep, '/')}"
                images.append(web_path)
    insect_images[species] = sorted(images)

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🐞 Insect Carousel", style={"textAlign": "center", "marginBottom": "25px"}),

    html.Div([
        html.Label("Select insect species:", style={
            "fontWeight": "normal", "fontSize": "15px", "marginBottom": "8px"
        }),
        dcc.Dropdown(
            id='species-dropdown',
            options=[{"label": s.replace("_", " ").capitalize(), "value": s} for s in INSECT_SPECIES],
            value=INSECT_SPECIES[0] if INSECT_SPECIES else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])


# 🔁 Carousel update
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value')
)
def update_carousel(species):
    images = insect_images.get(species, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "5px 10px", "marginRight": "15px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "430px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "5px 10px", "marginLeft": "15px"}),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontStyle": "italic"})
    ])

# 🔁 Navigation buttons
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Launch
clear_output(wait=True)
app.run(port=8454)
